In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from modules.objects import DataframeColumnSelector
import statsmodels.api as sm
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import set_config

df_train = pd.read_csv('raw_data/bb_train.csv')
# Drop the null bb_type entries as there are only six
df_train = df_train[df_train.bb_type.notnull()].copy()
df_dimension = pd.read_csv('raw_data/park_dimensions.csv')

In [2]:
features_to_use =  [
                'is_batter_lefty',
                'is_pitcher_lefty',
                'bb_type',
                'bearing',
                'pitch_name',
                'inning',
                'outs_when_up',
                'balls',
                'strikes',
                'plate_x',
                'plate_z',
                'pitch_mph',
                'launch_speed',
                'launch_angle',
                'is_home_run']

continuous_features = ['launch_angle',
                       'pitch_mph',
                       'plate_x',
                       'plate_z',
                       'launch_speed']

categorical_features = [i for i in features_to_use if (i != "is_home_run" and i not in continuous_features)]

categorical_transformers = []
continuous_transformers = []

for categorical_feature in categorical_features:

    ohe = OneHotEncoder()

    categorical_feature_pipeline = Pipeline(
        steps = [
            ('one_hot_encoder',ohe)
                ]
                                            )       
    categorical_transformers.append((
            f"{categorical_feature}_pipeline",
            categorical_feature_pipeline,
            [categorical_feature]))   

for continuous_feature in continuous_features:

    if continuous_feature not in ['launch_angle','launch_speed']:

        scaler = StandardScaler()

        numeric_feature_pipeline = Pipeline(
            steps = [
                ('scaler',scaler)
            ]
        )

        continuous_transformers.append((
            f"{continuous_feature}_pipeline",
            numeric_feature_pipeline,
            [continuous_feature]))
    
    else:
    
        scaler = StandardScaler()
        imputer = IterativeImputer()

        numeric_feature_pipeline = Pipeline(
            steps = [
                ('scaler',scaler),
                ('imputer',imputer)])

        continuous_transformers.append((
            f"{continuous_feature}_pipeline",
            numeric_feature_pipeline,
            [continuous_feature]))

all_transformers = categorical_transformers + continuous_transformers
data_preprocessor = ColumnTransformer(all_transformers,remainder='passthrough')

transformation_pipeline = Pipeline(
    steps = [
        ('column_selector',DataframeColumnSelector(col_list=features_to_use)),
        ('preprocessor',data_preprocessor)
            ])
set_config(display="diagram")

transformation_pipeline

Pipeline(steps=[('column_selector',
                 DataframeColumnSelector(col_list=['is_batter_lefty',
                                                   'is_pitcher_lefty',
                                                   'bb_type', 'bearing',
                                                   'pitch_name', 'inning',
                                                   'outs_when_up', 'balls',
                                                   'strikes', 'plate_x',
                                                   'plate_z', 'pitch_mph',
                                                   'launch_speed',
                                                   'launch_angle',
                                                   'is_home_run'])),
                ('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('is_batter_lefty_...
                                                 ('pitch_mph_pipeline',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['pitch_mph']),
                                                 ('plate_x_pipeline',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['plate_x']),
                                                 ('plate_z_pipeline',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['plate_z']),
                                                 ('launch_speed_pipeline',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('imputer',
                                                                   IterativeImputer())]),
                                                  ['launch_speed'])]))])

In [3]:
# Transform data
to_logreg_data = transformation_pipeline.fit_transform(df_train)

# Instantiate an empty list to extract column names in to
all_column_names = []

# Iterate through steps of the pipeline
for i in range(len(transformation_pipeline.named_steps['preprocessor'].transformers_)):

    try: # If the "step" is a transformer, then get the feature names
        all_column_names.extend([j for j in transformation_pipeline.named_steps['preprocessor'].transformers_[i][1].get_feature_names_out()])
    except AttributeError as e: 
        try:# Otherwise if it's a pipeline (imputer first), grab the second element's names
            all_column_names.extend(
                [i for i in transformation_pipeline.named_steps['preprocessor'].transformers_[i][1][0].get_feature_names_out()]
                )
        except AttributeError as e: # We're dealing with a string at this point
            all_column_names.append('is_home_run')

# Coerce from CSR matrix to pd.DataFrame
transformed_data = pd.DataFrame(to_logreg_data,columns=all_column_names)

# Manually coerce columns to integers where applicable
for column_name in [i for i in transformed_data.columns if i not in continuous_features]:

    transformed_data[column_name] = transformed_data[column_name].astype(int)

transformed_data

,is_batter_lefty_0,is_batter_lefty_1,is_pitcher_lefty_0,is_pitcher_lefty_1,bb_type_fly_ball,bb_type_ground_ball,bb_type_line_drive,bb_type_popup,bearing_center,bearing_left,...,balls_3,strikes_0,strikes_1,strikes_2,launch_angle,pitch_mph,plate_x,plate_z,launch_speed,is_home_run
0,1,0,0,1,0,0,1,0,0,1,...,0,0,0,1,-1.203632e-01,-0.179029,-0.265597,-0.618638,1.410355e+00,0
1,1,0,0,1,0,0,1,0,0,1,...,1,0,1,0,1.911520e-01,1.070091,-1.297363,0.973815,5.443491e-16,0
2,1,0,0,1,0,0,0,1,0,1,...,0,0,0,1,1.264149e+00,-0.398469,-0.984149,-1.003627,-8.814336e-01,0
3,0,1,0,1,1,0,0,0,0,0,...,0,0,0,1,1.437213e+00,-0.364709,-0.763056,1.446302,1.731950e-01,0
4,0,1,0,1,0,1,0,0,0,1,...,0,1,0,0,-2.242017e-01,1.222011,0.342408,1.656295,5.443491e-16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46233,0,1,1,0,0,0,1,0,0,0,...,0,0,0,1,4.680545e-01,-1.343749,0.471378,0.851319,7.343115e-01,0
46234,0,1,1,0,1,0,0,0,0,1,...,0,0,1,0,7.795698e-01,-2.778549,-1.020997,-0.496142,3.895290e-01,0
46235,1,0,0,1,0,0,1,0,1,0,...,0,1,0,0,2.887211e-17,1.492091,0.305559,1.358804,1.187261e+00,0
46236,0,1,0,1,0,0,1,0,0,1,...,0,0,0,1,5.270080e-02,-0.837349,0.489803,0.921317,7.545928e-01,0


In [7]:
testing_df = pd.read_csv('raw_data/bb_test.csv')
testing_df = testing_df.loc[testing_df.bb_type.notnull()]

transformation_pipeline.transform(testing_df)

KeyError: "['is_home_run'] not in index"